In [1]:
import itertools
import sys, os

import numpy as np
import pandas as pd
from scipy.special import comb
from scipy import stats
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

sns.set(rc={'figure.figsize':(15,8)})
sns.set_context('poster')

In [2]:
%load_ext autoreload
%autoreload 2
#import CCPA_lib as cp
import sorf_lib as sf

In [3]:
sns.set_context('poster')

Prochlorococcus marinus str. MIT 9313, complete genome
https://www.ncbi.nlm.nih.gov/nuccore/NC_005071.1/

    
Prochlorococcus marinus MIT9313 complete genome
https://www.ncbi.nlm.nih.gov/nuccore/BX548175




![](http://oregonstate.edu/instruct/bb450/fall14/stryer7/2/table_02_02.jpg)

https://www.uniprot.org/uniprot/Q7V735

http://tigrfams.jcvi.org/cgi-bin/HmmReportPage.cgi?acc=TIGR03798

https://www.ebi.ac.uk/training/online/course/interpro-functional-and-structural-analysis-protei/sequence-searching/searching-interpro-batc

http://www.ebi.ac.uk/interpro/sequencesearch/iprscan5-S20190707-131508-0462-76111813-p1m

https://www.ebi.ac.uk/Tools/services/rest/iprscan5/result/iprscan5-S20190707-131508-0462-76111813-p1m/json

In [4]:
genome='MIT9313'

In [5]:
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqIO

In [6]:
# get all sequence records for the specified genbank file
recs = [rec for rec in SeqIO.parse(os.path.join('data', "MIT9313.gb"), "genbank")]

# print the number of sequence records that were extracted
len(recs)


1

In [7]:
int(recs[0].features[1].location.end)

1340

In [8]:
    merge_cols = ['contig_id', 'gene_id', 'location', 'type', 
                  'start', 'stop', 'strand',
                  'function', 'figfam',
                  'nucleotide_sequence', 'aa_sequence', 'genome', 'left', 'right']


In [9]:

def gb_to_record(i, genome='MIT9313'):
    q = i.qualifiers
    g = lambda x : ','.join(q.get(x, []))
    pmtid = [i for i in q.get('old_locus_tag',[]) if i.startswith('PMT')]
    record = {
        'contig_id' : genome, 
        'gene_id' : g('locus_tag'), 
        'pmt_id': ','.join(pmtid), 
        'type' : i.type, 
        'location' : g('locus_tag'),  
        'strand' : '+' if i.location.strand == 1 else '-',
        'start' : int(i.location.start),
        'stop' : int(i.location.end),
        'left' : int(i.location.start),
        'right' : int(i.location.end),
        'function' : g('product'), 
        'genome' : genome, 
        'old_locus_tag' : g('old_locus_tag'), 
        'product' : g('product'), 
        'db_xref' : g('db_xref'), 
        'protein_id' : g('protein_id'), 
        'figfam' : '',
        'nucleotide_sequence' : '', 
        'aa_sequence' : g('translation'), 
    }
    return record


In [10]:
types_to_collect = ['CDS', 'ncRNA', 'rRNA', 'regulatory', 'tRNA', 'tmRNA']
records = [gb_to_record(i) for i in recs[0].features if i.type in types_to_collect]

In [11]:
gdf = pd.DataFrame(records)

In [12]:

gdf.to_feather(os.path.join('data', f'{genome}.ncbi.gb.feather.gz'))


In [13]:
gdf.loc[gdf.type != 'CDS'].head(20)

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence
44,MIT9313,AKG35_RS00225,PMT_RNA_44,tRNA,AKG35_RS00225,-,42281,42353,42281,42353,tRNA-Gly,MIT9313,"PMT_RNA_44,RG24_RS00225,RNA_44",tRNA-Gly,,,,,
84,MIT9313,AKG35_RS00430,PMT_RNA_1,tRNA,AKG35_RS00430,+,87819,87891,87819,87891,tRNA-Gly,MIT9313,"PMT_RNA_1,RG24_RS00430,RNA_1",tRNA-Gly,,,,,
162,MIT9313,,,regulatory,,+,178355,178459,178355,178459,,MIT9313,,,RFAM:RF00059,,,,
187,MIT9313,AKG35_RS00920,,rRNA,AKG35_RS00920,+,208846,210328,208846,210328,16S ribosomal RNA,MIT9313,"RG24_RS00925,RNA_53",16S ribosomal RNA,,,,,
188,MIT9313,AKG35_RS00925,PMT_RNA_2,tRNA,AKG35_RS00925,+,210510,210584,210510,210584,tRNA-Ile,MIT9313,"PMT_RNA_2,RG24_RS00930,RNA_2",tRNA-Ile,,,,,
189,MIT9313,AKG35_RS00930,PMT_RNA_3,tRNA,AKG35_RS00930,+,210593,210666,210593,210666,tRNA-Ala,MIT9313,"PMT_RNA_3,RG24_RS00935,RNA_3",tRNA-Ala,,,,,
190,MIT9313,AKG35_RS00940,,rRNA,AKG35_RS00940,+,211155,214034,211155,214034,23S ribosomal RNA,MIT9313,"RG24_RS00945,RNA_54",23S ribosomal RNA,,,,,
191,MIT9313,AKG35_RS00945,,rRNA,AKG35_RS00945,+,214141,214258,214141,214258,5S ribosomal RNA,MIT9313,"RG24_RS00950,RNA_55",5S ribosomal RNA,RFAM:RF00001,,,,
207,MIT9313,AKG35_RS01015,PMT_RNA_43,tRNA,AKG35_RS01015,-,227744,227816,227744,227816,tRNA-Thr,MIT9313,"PMT_RNA_43,RG24_RS01020,RNA_43",tRNA-Thr,,,,,
208,MIT9313,AKG35_RS01020,PMT_RNA_42,tRNA,AKG35_RS01020,-,227826,227908,227826,227908,tRNA-Tyr,MIT9313,"PMT_RNA_42,RG24_RS01025,RNA_42",tRNA-Tyr,,,,,


In [14]:
genome = 'MIT9313'
accession = sf.get_accession(genome)
accession

'BX548175'

In [15]:
orf_dpath = 'orffinder_tmp'
orf_fpath = os.path.join(orf_dpath, f'{accession}.orffinder.fa')
df = sf.parse_orffinder_fasta(orf_fpath)

In [16]:
df['rast_left'] = df['left']
df['rast_right'] = df['right'] + 1

In [17]:
gdf.loc[gdf.location == 'AKG35_RS00020']

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence
3,MIT9313,AKG35_RS00020,"PMT0004,PMT_0004",CDS,AKG35_RS00020,+,4621,6079,4621,6079,amidophosphoribosyltransferase,MIT9313,"PMT0004,PMT_0004,RG24_RS00020",amidophosphoribosyltransferase,,WP_011129383.1,,,MCGIVGIVSTALVNQQIYDSLLLLQHRGQDSTGIATMDGSVFHLHK...


In [18]:
df.loc[(df.rast_left > 4620) & (df.rast_left < 4630),  ].T

,5047
orfid,ORF5048_BX548175.1:4621:6078
contig,BX548175.1
left,4621
right,6078
start,4621
stop,6078
strand,+
aaseq,MCGIVGIVSTALVNQQIYDSLLLLQHRGQDSTGIATMDGSVFHLHK...
len_nn,1458
len_aa,485


In [19]:
df = sf.add_gene_annotation_to_sorf_df(df, gdf)
df = sf.add_overlapping_genes(df, gdf)

In [20]:
summary_df = sf.gen_summary_df(df, genome)
df.to_feather(os.path.join(orf_dpath, f'{genome}.ncbi.sorf.feather.gz'))
summary_df.to_csv(os.path.join(orf_dpath, f'{genome}.ncbi.sorf.summary.csv.gz'))


In [21]:
summary_df

,index,otype,count,subset,genome
0,0,antisense,13432,all,MIT9313
1,1,in_frame,1763,all,MIT9313
2,2,known,1476,all,MIT9313
3,3,out_frame,8544,all,MIT9313
4,4,overlap,1488,all,MIT9313
5,5,standalone,4627,all,MIT9313
6,0,antisense,13005,<100aa,MIT9313
7,1,in_frame,1288,<100aa,MIT9313
8,2,known,307,<100aa,MIT9313
9,3,out_frame,8443,<100aa,MIT9313


In [22]:
orfid = 'ORF16879_BX548175.1:1658121:1657948'
orfid = 'ORF19962_BX548175.1:317208:317134'

In [23]:
df.loc[df.orfid.isin([orfid])].T

,19961
orfid,ORF19962_BX548175.1:317208:317134
contig,BX548175.1
left,317134
right,317208
start,317208
stop,317134
strand,-
aaseq,MIMRCDDNLNQTLLEEMKELGERQ
len_nn,75
len_aa,24


In [24]:
gdf.columns

Index(['contig_id', 'gene_id', 'pmt_id', 'type', 'location', 'strand', 'start',
       'stop', 'left', 'right', 'function', 'genome', 'old_locus_tag',
       'product', 'db_xref', 'protein_id', 'figfam', 'nucleotide_sequence',
       'aa_sequence'],
      dtype='object')

In [25]:
gdf.loc[(gdf.left > 317100) & (gdf.right < 317220)]

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence


In [26]:
gdf.loc[gdf['product'].str.contains('hypothetical protein')]

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence
1,MIT9313,AKG35_RS00010,"PMT0002,PMT_0002",CDS,AKG35_RS00010,+,1343,2120,1343,2120,hypothetical protein,MIT9313,"PMT0002,PMT_0002,RG24_RS00010",hypothetical protein,,WP_041384224.1,,,MNLPDQILLSDLLHHRVRCDQGLDHGPGVLPWMHPPVHRLLGWVSR...
5,MIT9313,AKG35_RS00030,"PMT0006,PMT_0006",CDS,AKG35_RS00030,-,8681,9575,8681,9575,hypothetical protein,MIT9313,"PMT0006,PMT_0006,RG24_RS00030",hypothetical protein,,WP_011129385.1,,,MKLNQPRYKPLNGWLRGAVLGLITTIVATPPTHALIPYVYEPSPLE...
7,MIT9313,AKG35_RS00040,"PMT0008,PMT_0008",CDS,AKG35_RS00040,+,10570,11260,10570,11260,hypothetical protein,MIT9313,"PMT0008,PMT_0008,RG24_RS00040",hypothetical protein,,WP_080502855.1,,,MITFWHRALPASELGSSSAGRLTPLMRWLGLTLVILLLLQMVVLLG...
33,MIT9313,AKG35_RS00170,PMT_2275,CDS,AKG35_RS00170,+,36229,36442,36229,36442,hypothetical protein,MIT9313,"PMT_2275,RG24_RS00170",hypothetical protein,,WP_041384227.1,,,MMRWLLLGLLLFGLGTGLRNGWLVVHWSQFLRDAGLTFIDPEKPFN...
35,MIT9313,AKG35_RS00180,"PMT0035,PMT_0035",CDS,AKG35_RS00180,+,37342,37588,37342,37588,hypothetical protein,MIT9313,"PMT0035,PMT_0035,RG24_RS00180",hypothetical protein,,WP_011129414.1,,,MYYILGLVMASSYLRASAISVSMLPRRACPRIVQGRQISSAFHFVL...
38,MIT9313,AKG35_RS00195,,CDS,AKG35_RS00195,-,39717,40080,39717,40080,hypothetical protein,MIT9313,RG24_RS00195,hypothetical protein,,WP_041384825.1,,,MSGLLLSGLSCNSRSAPTNPSAESNISSQACLENLDLKGLDQALLR...
39,MIT9313,AKG35_RS00200,"PMT0038,PMT_0038",CDS,AKG35_RS00200,-,40108,40555,40108,40555,hypothetical protein,MIT9313,"PMT0038,PMT_0038,RG24_RS00200",hypothetical protein,,WP_011129417.1,,,MNRNRNRVALSLTLLGLVGISLSATAAQSATTDAASKGAQIYCFMR...
42,MIT9313,AKG35_RS00215,"PMT0041,PMT_0041",CDS,AKG35_RS00215,+,41651,41819,41651,41819,hypothetical protein,MIT9313,"PMT0041,PMT_0041,RG24_RS00215",hypothetical protein,,WP_157859756.1,,,MRSSIRLQAFHEGGHQLEKLEFALAVAATRGDQTRASVLRAQIETM...
43,MIT9313,AKG35_RS00220,"PMT0042,PMT_0042",CDS,AKG35_RS00220,+,41968,42178,41968,42178,hypothetical protein,MIT9313,"PMT0042,PMT_0042,RG24_RS00220",hypothetical protein,,WP_036913140.1,,,MRTTAGTNSASIQREVYLKAASGFFDRASAQAEAGDFQAAGSLILK...
48,MIT9313,AKG35_RS00245,"PMT0046,PMT_0046",CDS,AKG35_RS00245,+,46882,47494,46882,47494,hypothetical protein,MIT9313,"PMT0046,PMT_0046,RG24_RS00245",hypothetical protein,,WP_011129425.1,,,MTVAHQQDIQLDRRLQQDSIQFAGKTVYINPFLYWRRFDSNTDRWL...


In [27]:
gdf.loc[gdf.protein_id.str.contains('WP_011129965')]

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence
622,MIT9313,AKG35_RS13290,PMT_0586,CDS,AKG35_RS13290,-,643928,644084,643928,644084,hypothetical protein,MIT9313,PMT_0586,hypothetical protein,,WP_011129965.1,,,MAKRRNLKKEKQERNRSYARKFKKRKLRNDGRGEGAGNGVTGTANN...


In [28]:
df.loc[df.orfid== 'ORF30023_BX548175.1:644083:643928']

,orfid,contig,left,right,start,stop,strand,aaseq,len_nn,len_aa,...,overlap_strand,overlap_gene_type,overlap_count,is_same_strand,is_out_of_frame,is_inside,is_upstream,is_downstream,overlap_type,otype
30022,ORF30023_BX548175.1:644083:643928,BX548175.1,643928,644083,644083,643928,-,MAKRRNLKKEKQERNRSYARKFKKRKLRNDGRGEGAGNGVTGTANN...,156,51,...,-,CDS,1,NaN,NaN,NaN,NaN,NaN,known,known


In [29]:
summary_df

,index,otype,count,subset,genome
0,0,antisense,13432,all,MIT9313
1,1,in_frame,1763,all,MIT9313
2,2,known,1476,all,MIT9313
3,3,out_frame,8544,all,MIT9313
4,4,overlap,1488,all,MIT9313
5,5,standalone,4627,all,MIT9313
6,0,antisense,13005,<100aa,MIT9313
7,1,in_frame,1288,<100aa,MIT9313
8,2,known,307,<100aa,MIT9313
9,3,out_frame,8443,<100aa,MIT9313


In [30]:
gdf.left.value_counts()

727039     1
1376524    1
511319     1
27989      1
929743     1
          ..
1585853    1
371388     1
1489397    1
1442487    1
813617     1
Name: left, Length: 2459, dtype: int64

In [31]:
gdf.head()

,contig_id,gene_id,pmt_id,type,location,strand,start,stop,left,right,function,genome,old_locus_tag,product,db_xref,protein_id,figfam,nucleotide_sequence,aa_sequence
0,MIT9313,AKG35_RS00005,"PMT0001,PMT_0001",CDS,AKG35_RS00005,+,173,1340,173,1340,DNA polymerase III subunit beta,MIT9313,"PMT0001,PMT_0001,RG24_RS00005",DNA polymerase III subunit beta,,WP_011129380.1,,,MKLVCSQAELNTALQLVSRAVASRPTHPVLANVLLTADAGTDRLSL...
1,MIT9313,AKG35_RS00010,"PMT0002,PMT_0002",CDS,AKG35_RS00010,+,1343,2120,1343,2120,hypothetical protein,MIT9313,"PMT0002,PMT_0002,RG24_RS00010",hypothetical protein,,WP_041384224.1,,,MNLPDQILLSDLLHHRVRCDQGLDHGPGVLPWMHPPVHRLLGWVSR...
2,MIT9313,AKG35_RS00015,"PMT0003,PMT_0003",CDS,AKG35_RS00015,+,2177,4562,2177,4562,phosphoribosylformylglycinamidine synthase sub...,MIT9313,"PMT0003,PMT_0003,RG24_RS00015",phosphoribosylformylglycinamidine synthase sub...,,WP_011129382.1,,,MRVDYDVAAALRHEGLKPHDYDEICRRLQRAPNRVELGMFGVMWSE...
3,MIT9313,AKG35_RS00020,"PMT0004,PMT_0004",CDS,AKG35_RS00020,+,4621,6079,4621,6079,amidophosphoribosyltransferase,MIT9313,"PMT0004,PMT_0004,RG24_RS00020",amidophosphoribosyltransferase,,WP_011129383.1,,,MCGIVGIVSTALVNQQIYDSLLLLQHRGQDSTGIATMDGSVFHLHK...
4,MIT9313,AKG35_RS00025,"PMT0005,PMT_0005",CDS,AKG35_RS00025,-,6114,8604,6114,8604,DNA topoisomerase 4 subunit A,MIT9313,"PMT0005,PMT_0005,RG24_RS00025",DNA topoisomerase 4 subunit A,,WP_011129384.1,,,MAEERLQPIALHQEMQRSYLEYAMSVIVGRALPDARDGLKPVQRRI...


In [32]:
gdf.type.value_counts()

CDS           2403
tRNA            44
rRNA             6
ncRNA            3
regulatory       2
tmRNA            1
Name: type, dtype: int64

In [34]:
df.loc[df.orfid == 'ORF3795_BX548175.1:1742715:1742936',
[
    'left', 'left_overlap', 'right', 'right_overlap', 'rast_right', 'rast_left','overlap_type' ,
    'start', 'start_overlap', 'strand',
    'aaseq', 'aa_sequence_overlap', 'otype'
]
                  
                 ].T

,3794
left,1742715
left_overlap,NaN
right,1742936
right_overlap,NaN
rast_right,1742937
rast_left,1742715
overlap_type,upstream
start,1742715
start_overlap,NaN
strand,+
